# Converting the Mathematical Basis of a 3D Graphics Engine into Code #

## Introduction ##
### Justification ###
In my previous exploration; I designed a the mathematical basis for a simple 3D graphics engine. Given the coordinates of the camera, the orientation of the camera, the aspect ratio of the camera, and the coordinates of a certain point in space; this engine can theoretically produce the coordinates of the point in space in 2D relative to the camera's point of view. In the conclusion of that investigation; I stated that a logical extension would be to convert the mathematical basis outlined there into code that could visually display this translation between dimensions. This exploration documents that logical extension.

### Tools ###
I was originally planning on using Python; but I am so familiar with it at this point and I want to learn something new. I started learning Julia a while ago and thoroughly enjoyed its syntax. Julia is similar to Python in many ways; but differs in that it is faster; especially when dealing with mathematics. I believe that Julia has tremendous potential and I therefore want to learn it early to gain a competitive edge.

I was also planning on using Pygame at first; but I obviously cannot easily integrate Julia with Pygame, so I'll have to use a library called GameZero.jl, which is not quite as sophisticated as Pygame, but works well enough for what we are trying to accomplish here.

## Math Section ##

### Packages ###
For this project, I am using GenericLinearAlgebra.jl. Constructing a 3D graphics engine obviously requires at least some linear algebra, and this is the best Julia library that I could find for the job.

In [ ]:
using GenericLinearAlgebra
const GenericLinearAlgebra = LA

### Defining Variables ###
First, we need to define the variables concerning the camera. These are the position of the camera, its orientation, and its aspect ratio. All of these variables are mutable structs because their values are subject to change should we decide to introduce animation into this program. Dictionaries could have accomplished the same task, but apparently you are only supposed to use dictionaries in Julia if you don't know what the keys are going to be in advance. Mutable structs are more efficient otherwise.

In [6]:


Point = Vector{Float64}(undef, 3)

mutable struct Orientation
    Yaw::Float64
    Pitch::Float64
    Roll::Float64
end

mutable struct AspectRatio
    Width :: Int64
    Height :: Int64
end

mutable struct Camera
    Coords::Point
    Orientation::Orientation
    AspectRatio::AspectRatio
end

### Line of Sight ###
Next, we should write a function that, given information about the camera, can return a function for the camera's line of sight. Recall that the line of sight function was as follows:

 $cam(a) = \begin{bmatrix} (x_2 - p_x)a + p_x \\  (y_2 - p_y)a + p_y \\ (z_2 - p_z)a + p_z\end{bmatrix}$

However, before we can write this function, we must determine the value of $p_2=\begin{bmatrix} x_2 \\ y_2 \\ z_2\end{bmatrix}$. $p_1$ (the camera's position) and $p_2$ form the unit vector for the line of sight. In other words, the distance between $p_1$ and $p_2$ is one unit. In code form, we'll refer to $p_2$ as `unitVec`. Recall that we can find the value of `unitVec` using rotation matrices:

$p_2 = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}\begin{bmatrix}\cos{\theta_{y}} & 0 & -\sin{\theta_{y}} \\ 0 & 1 & 0 \\ \sin{\theta_{y}} & 0 & \cos{\theta_{y}}\end{bmatrix}\begin{bmatrix}1 & 0 & 0 \\ 0 & \cos{\theta_{p}} & -\sin{\theta_{p}} \\ 0 & \sin{\theta_{p}} & \cos{\theta_{p}}\end{bmatrix}+p$

In [ ]:
function findUnitVec(Coords::Point, Orientation::Orientation)
    yawRoation = [
        cos(Yaw) 0.0 -sin(Yaw);
           0.0   1.0   0.0    ;
        sin(Yaw) 0.0  cos(Yaw)
    ]
    pitchRotation = [
        1.0     0.0          0.0  ;
        0.0 cos(Pitch) -sin(Pitch);
        0.0 sin(Pitch)  cos(Pitch) 
    ]
    Init = [0.0, 0.0, 1.0]
    return Init * yawRotation * pitchRotation + Coords
end

Now that we have a way of finding `unitVec`, we can write a function that finds the line of sight of the camera.

In [ ]:

function lineOfSightGenerator(Coords::Point, unitVec::Point)
    function lineOfSight(a::Float64)
        axis = Symbol(axis)
        return (unitVec - Coords) * LA.UniformScaling(a) + Coords
    end
    return lineOfSight
end

### The Orthogonal Plane ###
Next, we need to write the function which, given a point in space, determines the $a$ value which forms a plane that contains $cam(a)$ and that point in space, and is also orthogonal to the line of sight. The equation for this $a$ value is as follows:

$a = (x_2 - p_x)e_x + (y_2 - p_y)e_y + (z_2 - p_z)e_z + p_x^2 + p_y^2 + p_z^2 - x_2p_x - y_2p_y - x_2p_z$

In writing the code for this formula, I realized that I had made a major oversight in my original formula, which just goes to show the importance of implementing your theories in a real model to ensure that they are mathematically and logically consistent.

In [ ]:
function getA(Coords::Point, Point::Point, unitVec::Point)
    return (unitVec[1] - Coords[1]) * Point[1]
        + (unitVec[2] - Coords[2]) * Point[2]
        + (unitVec[3] - Coords[3]) * Point[3]
        + square(Coords[1])
        + square(Coords[2])
        + square(Coords[3])
        - Coords[1] * unitVec[1]
        - Coords[2] * unitVec[2]
        - Coords[3] * unitVec[3]
end

### Distance to the Corners ###
Now that we have a way of determining the $a$ value for any point in space (including negative $a$ values, which I will discuss later), we can determine the distance from $cam(a)$ to any corner of the frame. Recall that the equation for this is as follows, where $\phi_x$ and $\phi_y$ are `AspectRatio.Width` and `AspectRatio.Height`, respectively:

$cornerdist(a, \phi_x, \phi_y)=\sqrt{(a\cos{\phi_x})^2 + (a\cos{\phi_y})^2}$

In [ ]:
function cornerDist(a::Float64, AspectRatio::AspectRatio)
    return sqrt(
        square(a * cos(Camera.AspectRatio.Width))
        + square(a * cos(Camera.AspectRatio.Height))
        )
end

### Writing the Circle Formula ###
A 3D circle formula (not a sphere, but a flat circle in 3D space) is helpful in this case because every rectangle can circumscribed. Thus, no matter how much the camera rolls, there will always be a circle which contains all four of its frame points for any $a$ value. By changing the angle that we give the formula, we can find the exact positions of all four frame points for any $a$ value. To write the circle formula, we need two orthogonal unit vectors which are also both orthogonal to the camera's line. We also need the radius, but we have already solved for this with `cornerDist`. 

$v_{z0} = \frac{-a_y - a_z}{a_z}$

The magnitude of the first orthogonal vector $v_0$ must be equal to 1, so we can solve for a scalar that satisfies the equation:

$ \vec{v} = {\lVert \begin{bmatrix} 1 \\ 1 \\ v_z0 \end{bmatrix} \rVert}^{-1} \begin{bmatrix} 1 \\ 1 \\ v_z0 \end{bmatrix}$

In [ ]:
function V_z0(unitVec::Point)
    return (-unitVec[2] - unitVec[1]) / unitVec[3]
end

function V(v_z0::Point)
    Unscaled = Vector([1, 1, v_z0])
    return (LA.norm(Unscaled) ^ -1) * Unscaled
end

The vector that is orthogonal to both the camera line and $\vec{v}$, $\vec{v}'$, can be found with the following formula. Note that the $\times$ operator refers to the cross product operation rather than multiplication.

$\vec{v} \times \vec{a} = \vec{v'_0}$

$\vec{v'} = {\lVert \vec{v'_0} \rVert}^{-1} \vec{v'_0}$

In [ ]:
function V'(v::Point, unitVec::Point)
    v_0' = cross(v, unitVec)
    return (LA.norm(v_0') ^ -1) * v_0'
end

Now that we have a way of finding $\vec{v}$ and $\vec{v}'$, we can write the circle equation. The formula for the circle is as follows, where $r$ is the radius:

$circle(\gamma) = cam(a) + rv\sin{\gamma} + rv'\cos{\gamma}$

In [ ]:
function circleGenerator(Point::Point, a::Point, v::Point, v'::Point, Cam::Function, r::Float64)
    function Circle(gamma::Int64)
        return Cam(a) + LA.UniformScaling(r) * (LA.UniformScaling(sin(gamma)) * v + LA.UniformScaling(cos(gamma)) * v')
    end
    return Circle
end

### Determining $\gamma_0$ ###
$\gamma_0$ is the angle which, when plugged into the $circle$ function (which will I will write and explain later) produces a point which satisfies the following system (with angles in degrees):

$circle(\gamma_0 - 1)_y < circle(\gamma_0)_y < circle(\gamma_0 + 1)_y$ 

$circle(\gamma_0)_y = cam(a)_y$

The following formula produces one value, but we can derive two values from it by adding $\pi$ to the original value.

$\gamma_0 = \tan^{-1}{\frac{-v'_y}{v_y}}$

Only one of the two values will satisfy the first equation in the system. This is the real $\gamma_0$.

In [ ]:
function Gamma_0(v::Point, v'::Point):
    return atan(-(v'[2]), v[2])
end

### Determining Frame Points ###
Frame corners are numbered 1-4, starting at the top right and rotating clockwise.

In [ ]:
function Y_theta(AspectRatio::AspectRatio)
    l_x = AspectRatio.Width / 2
    l_y = AspectRatio.Height / 2
    return atan(l_y, l_x)
end

function X_theta(y_theta::Float64)
    return pi / 2 - y_theta
end

function frameAngles(Orientation::Orientation, x_theta::Float64, y_theta::Float64, gamma_0::Float64)
    Roll = Orientation.Roll
    return [1 0 1; 1 1 1; 2 1 1; 2 2 1] * [y_theta; x_theta; gamma_0 - Roll]
end

function framePoints(Circle::Function, Angles::Vector)
    return map(Circle, Angles)
end

### Conversion to 2D ###
Now, we have a plane which contains the four frame points, the target point and the central point. We just need to convert this plane into two dimensions. The vector formed by frame points 3 and 4 represents the Y axis of the 2D plane, while the vector formed by frame points 3 and 2 represents the X axis of the 2D plane. The projections of the vector formed by frame point 3 and the target point onto both of the axes tells us whether the target point is contained in the canvas and, if so, the exact position on the canvas.

In [ ]:
function twoD_xAxis(Frame::Matrix)
    return Frame[4] - Frame[3]
end

function twoD_yAxis(Frame::Matrix)
    return Frame[2] - Frame[3]
end

function pointVector(Frame::Matrix, Point::Point)
    return Point - Frame[3]

function Proj(Axis::Vector, pointVector::Vector)
    return LA.dot(Axis, pointVector)
end

function getPointRatio(X::Vector, Y::Vector, xProj::Float64, yProj::Float64)
    xMag = LA.norm(X)
    yMag = LA.norm(Y)
    if 0 < xProj < xMag && 0 < yProj < yMag
        return (X = xProj / xMag, Y = yProj / yMag)
    else
        return false
    end
end

function ratioToCoords(Ratio::NamedTuple, AspectRatio::AspectRatio)
    if issubset([false], Ratio):
        return false
    end
    x = Ratio.X * AspectRatio.Width
    y = Ratio.Y * AspectRatio.Height
    return (X = x, Y = y)
end

### Gluing the Functions Together ###
Now, we have a bunch of functions. But we still don't have a 3D graphics engine. Let's fix this. We need to write a function which will group together the individual tasks that each function executes so that we can see the process through to the end. This process was a learning experience for me because there were two ways which I could have gone about combining the functions and I was not initially sure which method I should employ. The first method (Method A) involves computing all the variables which any given function needs in order to run within the lexical scope of that function. An example of Method A is shown below.

In [ ]:
#=
@memoize LRU(maxsize=1) function framePoints(Camera::Camera, Point::Point, Orientation::Orientation)
    Roll = Orientation.Roll
    Circle = circleGenerator(Camera, Point)
    Angles = frameAngles(Camera, Point, Roll)
    return map(Circle, Angles)
end
=#

Note that the only computation unique to `framePoints` is `map(Circle, Angles)` while the rest of the function is simply setup for this computation. Also note the `@memoize LRU(maxsize=1)` macro preceding the function. If we were to use Method A, almost all functions would be memoized so that if they were ever called in two different functions with the same arguments, the computation would not be redundant. An advantage of Method A is that all of the functions are "linked" together since they reference each other. This means that less work goes into combining all of those functions at the end.

Method B involves passing all (or almost all in some edge cases) of the variables required for any given function to run as parameters. This means the values for those variables must be computed outside of the lexical scope of the function. An example of Method B is shown below.

In [ ]:
#=
function framePoints(Circle::Function, Angles::Vector)
    return map(Circle, Angles)
end
=#

Note that there is much less setup for the function to be able to run. We also no longer need memoization as long as all of these parameters are computed and cached within the same lexical scope. By far the most salient advantage of Method B is the fact that it makes testing and debugging significantly easier since helper functions do not directly rely on each other. Each function can be tested in isolation, which makes finding most bugs a piece of cake. For this reason, I decided to employ Method B. There may be a time in the future where Method A makes more practical sense, but that time is not now.

As a consequence of employing Method B, the "glue" function is significantly longer.

In [ ]:
function three2Two(Camera::Camera, Point::Point)
    Orientation = Camera.Orientation
    AspectRatio = Camera.AspectRatio
    Coords = Camera.Coords
    unitVec = findUnitVec(Coords, Orientation)
    Cam = lineOfSightGenerator(Coords, unitVec)
    A = getA(Coords, Point, unitVec)
    r = cornerDist(A, AspectRatio)
    v_z0 = V_z0(unitVec)
    v = V(v_z0)
    v' = V'(v, unitVec)
    Circle = circleGenerator(Point, A, v, v', Cam, r)
    gamma_0 = Gamma_0(v, v')
    y_theta = Y_theta(AspectRatio)
    x_theta = X_theta(y_theta)
    Angles = frameAngles(Orientation, x_theta, y_theta, gamma_0)
    Frame = framePoints(Circle, Angles)
    X = twoD_xAxis(Frame)
    Y = twoD_yAxis(Frame)
    PV = pointVector(Frame, Point)
    xProj = Proj(X, PV)
    yProj = Proj(Y, PV)
    Ratio = getPointRatio(X, Y, xProj, yProj)
    return ratioToCoords(Ratio, AspectRatio)
end

# Implementation Section #


Now, we have a function, `three2Two`, which, given information about the camera and the point in 3D space, can give us the coordinates of that point on a 2D canvas. Next, we need to use this function to create a simple GUI so that users can actually visualize 3D-2D transformations. To do this, as mentioned previously, we will use a simple Julia GUI library called GameZero.jl.

We'll start by creating a blank canvas with a height of 400 px and a width of 644 pixels. Why 644? The golden ratio is ~1.61, and 400 * 1.61 = 644. Unfortunately, GameZero.jl  does not allow us to change the dimensions of the canvas after the program is initialized, so we'll just stick with these dimensions. We'll also use a white background just because I want users to develop early onset cataracts.

In [ ]:
HEIGHT = 400
WIDTH = 644
center = (w = int(round(WIDTH / 2)) , h = int(round(HEIGHT / 2)))
BACKGROUND = colorant"white"

Next, we'll create instances of the structures that we wrote at the beginning of this document. GameZero.jl is so limited in its capabilities that it does not even support text input. Therefore, we'll have to update these variables using keys like the up arrow, fore example.

In [ ]:
point = Point([0; 0; 0])
orientation = Orientation(Yaw = 0, Pitch = 0, Roll = 0)
aspectRatio = AspectRatio(Width = WIDTH, Height = HEIGHT)
camera = Camera(Coords = [0; 0; 1], Orientation = orientation, AspectRatio = aspectRatio)

In [ ]:
circle = Circle(center..., radius=5)
circle.pos = center
all_caught_up = false

function update(g::Game)
    if all_caught_up == true
        return
    end
    
    canvasCoords = three2Two(camera, point)
    if canvasCoords
        circle.pos = canvasCoords
    else
        # Move the circle off the canvas
        # I couldn't find a better way to do this unfortunately
        # GameZero is super barebones
        circle.pos = WIDTH + 100, HEIGHT + 100
    end
    
    two_degrees = deg2rad(2)
    keys_pressed = true

    # I wish I could find a more efficient way of doing this
    if g.keyboard.RIGHT
        orientation.Yaw -= two_degrees
    elseif g.keyboard.LEFT
        orienatation.Yaw += two_degrees
    elseif g.keyboard.UP
        orientation.Pitch -= two_degrees
    elseif g.keyboard.DOWN
        orientation.Pitch += two_degrees
    else
        keys_pressed = false
    end

    if keys_pressed
        all_caught_up = false
    end
end

# Testing #
Usually, I would have tests in a separate module, but I want all the code for this project to be accessible within a single document and therefore have decided to place them here.

In [ ]:
using Test

@testset "findUnitVec" begin
    xAxis = [2; 0; 0]
    yAxis = [0; 2; 0]
    zAxis = [0; 0; 2]
    noRot = Orientation(0, 0, 0)
    x90DegRot = Orientation(deg2rad(90), 0, 0)
    y90DegRot = Orientation(0, deg2rad(90), 0)
    z90DegRot = Orientation(0, 0, deg2rad(90))
    x45DegRot = Orientation(deg2rad(45), 0, 0)
    y45DegRot = Orientation(0, deg2rad(45), 0)
    z45DegRot = Orientation(0, 0, deg2rad(45))

    # Ensure that everything works when no rotation is done
    @test findUnitVec(xAxis, noRot) == [1; 0; 0]
    @test findUnitVec(yAxis, noRot) == [0; 1; 0]
    @test findUnitVec(zAxis, noRot) == [0; 0; 1]

    # Test 90 degree rotation
    @test findUnitVec(xAxis, x90DegRot) == [0; 0; -1]
    @test findUnitVec(yAxis, y90DegRot) == [0; 0; 1]
    @test findUnitVec(zAxis, z90DegRot) == [-1; 0; 0]

    # Test 45 degree rotation
    s2 = sqrt(2)
    @test findUnitVec(xAxis, x45DegRot) == [s2; 0; -s2]
    @test findUnitVec(yAxis, y45DegRot) == [0; s2; s2]
    @test findUnitVec(zAxis, z45DegRot) == [s2; 0; s2]

    # Ensure that outputs are in fact unit vectors
    @test findUnitVec([14.0, 19.0, 20.0], Orientation(2.33, 4.9, 40.0))
end

@testset "lineOfSightGenerator" begin
    @test
end